<a href="https://colab.research.google.com/github/PolinaBoeva/predictive_models_dota2/blob/main/notebooks/Best_checkpoint3_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Пайплайн обучения и
применения лучшей из моделей 3 чекпоинта

In [ ]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.preprocessing import StandardScaler

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
df_team_cleaned = pd.read_csv('df_team_cleaned.csv')
df_team_cleaned = df_team_cleaned.drop(df_team_cleaned.columns[0], axis=1)
df_team_cleaned.head()

,match_id,previous_kills_avr_team_1_mean,previous_kills_avr_team_1_max,previous_kills_avr_team_1_min,previous_hero_kills_avr_team_1_mean,previous_hero_kills_avr_team_1_max,previous_hero_kills_avr_team_1_min,previous_courier_kills_avr_team_1_mean,previous_courier_kills_avr_team_1_max,previous_courier_kills_avr_team_1_min,...,previous_win_avr_team_2_mean,previous_win_avr_team_2_max,previous_win_avr_team_2_min,previous_duration_avr_team_2_mean,previous_duration_avr_team_2_max,previous_duration_avr_team_2_min,previous_first_blood_time_avr_team_2_mean,previous_first_blood_time_avr_team_2_max,previous_first_blood_time_avr_team_2_min,radiant_win
0,6947109699,6.600000,11.0,2.0,6.600000,11.0,2.0,0.000000,0.000000,0.0,...,0.000000,0.0,0.000000,32.400000,32.400000,32.400000,2.066667,2.066667,2.066667,1
1,6947237459,9.400000,15.0,3.0,9.400000,15.0,3.0,0.600000,1.000000,0.0,...,0.000000,0.0,0.000000,37.366667,37.366667,37.366667,0.866667,0.866667,0.866667,1
2,6947793080,9.800000,24.0,3.0,9.800000,24.0,3.0,0.400000,2.000000,0.0,...,0.000000,0.0,0.000000,38.316667,38.316667,38.316667,1.833333,1.833333,1.833333,1
3,6947885612,7.000000,12.5,2.5,7.000000,12.5,2.5,0.200000,0.500000,0.0,...,1.000000,1.0,1.000000,33.883333,33.883333,33.883333,1.791667,1.791667,1.791667,1
4,6947952638,8.666667,15.0,4.0,8.666667,15.0,4.0,0.133333,0.666667,0.0,...,0.466667,1.0,0.333333,31.711111,31.711111,31.711111,1.933333,1.933333,1.933333,0


In [ ]:
y = df_team_cleaned['radiant_win']
X = df_team_cleaned.drop(columns=['radiant_win', 'match_id'])

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state = 42)

In [ ]:
def evaluate_model(y_test, predictions, X_test, model, model_name):
    global metrics_df
    accuracy = accuracy_score(y_test, predictions)
    precision = precision_score(y_test, predictions)
    recall = recall_score(y_test, predictions)
    f1 = f1_score(y_test, predictions)
    try:
        roc_auc = roc_auc_score(y_test, model.predict_proba(X_test)[:, 1])  # Для AUC-ROC используем predict_proba
    except ValueError:
        roc_auc = 0.5
    print(f"Metrics for {model_name}:")
    print(f"Accuracy: {round(accuracy, 3)}")
    print(f"Precision: {round(precision, 3)}")
    print(f"Recall: {round(recall, 3)}")
    print(f"F1 Score: {round(f1, 3)}")
    print(f"ROC-AUC: {round(roc_auc, 3)}")

    new_metrics = pd.DataFrame([[model_name, round(accuracy, 3), round(precision, 3),
                                 round(recall, 3), round(f1, 3), round(roc_auc, 3)]],
                                columns=metrics_df.columns)

    metrics_df = pd.concat([metrics_df, new_metrics], ignore_index=True)


In [ ]:
pip install catboost

In [ ]:
# catboost
from catboost import CatBoostClassifier

pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('model', CatBoostClassifier())
])

pipeline.fit(X_train, y_train)

In [ ]:
predictions = pipeline.predict(X_test)
catboost_metrics = evaluate_model(y_test, predictions, X_test, pipeline, 'CatBoost')
catboost_metrics

Metrics for CatBoost:
Accuracy: 0.601
Precision: 0.603
Recall: 0.616
F1 Score: 0.61
ROC-AUC: 0.655


Метрики модели CatBoost:

1. **Точность (Accuracy)**: 0.601  
   60.1% предсказаний верны.

2. **Точность (Precision)**: 0.603  
   Из всех предсказанных положительных классов, 60.3% действительно положительные.

3. **Полнота (Recall)**: 0.616  
   61.6% реальных положительных классов были правильно распознаны.

4. **F1-метрика**: 0.61  
   Гармоническое среднее точности и полноты. Хороший баланс между этими показателями.

5. **ROC-AUC**: 0.655  
   Модель имеет умеренную способность различать классы.

CatBoost показывает достаточно хорошие результаты по большинству метрик, особенно по recall и F1 Score, что говорит о том, что модель эффективно находит положительные примеры.